In [2]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

## Remote Inference via Inference Providers
Ensure you have a valid **HF_TOKEN** set in your environment. You can get your token from [your settings page](https://huggingface.co/settings/tokens). Note: running this may incur charges above the free tier.
The following Python example shows how to run the model remotely on HF Inference Providers, automatically selecting an available inference provider for you.
For more information on how to use the Inference Providers, please refer to our [documentation and guides](https://huggingface.co/docs/inference-providers/en/index).

In [5]:
import os
import json
os.environ['HF_TOKEN'] = 'YOUR_HF_TOKEN_HERE'

In [6]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="auto",
    api_key=os.environ["HF_TOKEN"],
)

def get_sentiment(text):
    """
    Get sentiment analysis for a given text using Hugging Face model.
    Returns the sentiment result or None if text is empty/None.
    """
    if not text or text.strip() == "":
        return None

    try:
        result = client.text_classification(text, model="nlptown/bert-base-multilingual-uncased-sentiment")
        return result
    except Exception as e:
        print(f"Error analyzing sentiment: {e}")
        return None
def analyze_pr_sentiments(json_file_path):
    """
    Read pull requests from JSON file and analyze sentiments.
    """
    # Read the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"Repository: {data['repository']}")
    print(f"Total PRs: {data['total_prs']}")
    print(f"Timestamp: {data['timestamp']}")
    print("\n" + "="*80 + "\n")

    # Analyze each pull request
    for pr in data['pull_requests']:
        print(f"PR #{pr['number']}: {pr['title']}")
        print(f"Author: {pr['author']}")
        print(f"State: {pr['state']}")
        print(f"URL: {pr['html_url']}")

        # Analyze PR title sentiment
        print("\n--- Title Sentiment ---")
        title_sentiment = get_sentiment(pr['title'])
        if title_sentiment:
            print(f"Title: {title_sentiment}")

        # Analyze PR body sentiment
        if pr['body']:
            print("\n--- Body Sentiment ---")
            # Truncate body if too long (models have token limits)
            body_text = pr['body'][:500]  # First 500 chars
            body_sentiment = get_sentiment(body_text)
            if body_sentiment:
                print(f"Body: {body_sentiment}")

        # Analyze comments sentiments
        if pr['comments'] and len(pr['comments']) > 0:
            print(f"\n--- Comments Sentiments ({pr['total_comments']} total) ---")
            for idx, comment in enumerate(pr['comments'], 1):
                if comment['body']:
                    # Truncate comment if too long
                    comment_text = comment['body'][:500]
                    comment_sentiment = get_sentiment(comment_text)
                    if comment_sentiment:
                        print(f"Comment {idx} by {comment['author']}: {comment_sentiment}")

        print("\n" + "="*80 + "\n")

def save_sentiments_to_file(json_file_path, output_file_path):
    """
    Read pull requests from JSON file, analyze sentiments, and save to a new JSON file.
    """
    # Read the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Analyze each pull request and add sentiment data
    for pr in data['pull_requests']:
        # Analyze PR title sentiment
        pr['title_sentiment'] = get_sentiment(pr['title'])

        # Analyze PR body sentiment
        if pr['body']:
            body_text = pr['body'][:500]
            pr['body_sentiment'] = get_sentiment(body_text)
        else:
            pr['body_sentiment'] = None

        # Analyze comments sentiments
        if pr['comments']:
            for comment in pr['comments']:
                if comment['body']:
                    comment_text = comment['body'][:500]
                    comment['sentiment'] = get_sentiment(comment_text)
                else:
                    comment['sentiment'] = None

    # Save to output file
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"Sentiment analysis complete! Results saved to {output_file_path}")

In [8]:
if __name__ == "__main__":
    # Input JSON file path
    input_file = "openai_evals_prs.json"

    # Output JSON file path (with sentiments)
    output_file = "prs_with_sentiments.json"

    # Option 1: Print sentiments to console
    print("Analyzing sentiments and printing to console...\n")
    analyze_pr_sentiments(input_file)

    # Option 2: Save sentiments to a new JSON file
    print("\nSaving sentiments to file...\n")
    save_sentiments_to_file(input_file, output_file)

Analyzing sentiments and printing to console...

Repository: openai/evals
Total PRs: 100
Timestamp: 2025-11-12T03:36:51.398133Z


PR #1605: Remove incontext_rl suite with defunct dependencies
Author: maxb-openai
State: closed
URL: https://github.com/openai/evals/pull/1605

--- Title Sentiment ---
Title: [TextClassificationOutputElement(label='1 star', score=0.5288680195808411), TextClassificationOutputElement(label='2 stars', score=0.20621776580810547), TextClassificationOutputElement(label='3 stars', score=0.13769695162773132), TextClassificationOutputElement(label='4 stars', score=0.06969050318002701), TextClassificationOutputElement(label='5 stars', score=0.057526715099811554)]

--- Body Sentiment ---
Body: [TextClassificationOutputElement(label='1 star', score=0.5151775479316711), TextClassificationOutputElement(label='2 stars', score=0.3456527590751648), TextClassificationOutputElement(label='3 stars', score=0.11431945860385895), TextClassificationOutputElement(label='4 stars', sc